In [ ]:
urls = input("Please enter the website of an Facebook post:")

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values':{'notifications': 2}}
options.add_experimental_option('prefs', prefs)

In [ ]:
username = input("Please enter your email or phone number:")
password = input("Please enter your password:")

s = Service('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')
driver = webdriver.Chrome(service = s, options = options)
driver.get('https://www.facebook.com/')
driver.find_element_by_id('email').send_keys(username)
driver.find_element_by_id('pass').send_keys(password)
driver.find_element_by_name('login').click()

In [ ]:
import pandas as pd

collect_comment = pd.DataFrame()
collect_fb = pd.DataFrame()

In [ ]:
# 導覽到指定貼文網址
driver.get(urls)

In [ ]:
# 將最相關留言改成所有留言
import time
from time import sleep

time.sleep(2)
driver.find_element_by_xpath("//div[@class = 'bp9cbjyn j83agx80 kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x h3fqq6jp']").click()
time.sleep(2)
driver.find_elements_by_xpath("//div[@class = 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz p7hjln8o esuyzwwr f1sip0of n00je7tq arfg74bv qs9ysxi8 k77z8yql abiwlrkh p8dawk7l lzcic4wl dwo3fsh8 rq0escxv nhd2j8a9 j83agx80 btwxx1t3 pfnyh3mw opuu4ng7 kj2yoqh6 kvgmc6g5 oygrvhab pybr56ya dflh9lhu f10w8fjw scb9dxdr l9j0dhe7 i1ao9s8h du4w35lb bp9cbjyn']")[2].click()
time.sleep(2)

In [ ]:
# 點開被隱藏的留言
for i in range(50):
    while(1):
        time.sleep(1)
        comment = driver.find_elements_by_xpath("//div[@class = 'oajrlxb2 g5ia77u1 mtkw9kbi tlpljxtp qensuy8j ppp5ayq2 goun2846 ccm00jje s44p3ltw mk2mc5f4 rt8b4zig n8ej3o3l agehan2d sk4xxmp2 rq0escxv nhd2j8a9 mg4g778l p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x tgvbjcpo hpfvmrgz jb3vyjys qt6c0cv9 a8nywdso l9j0dhe7 i1ao9s8h esuyzwwr f1sip0of du4w35lb n00je7tq arfg74bv qs9ysxi8 k77z8yql pq6dq46d btwxx1t3 abiwlrkh lzcic4wl bp9cbjyn m9osqain buofh1pr g5gj957u p8fzw8mz gpro0wi8']")
        if(len(comment) == 0):
            break
        for btn in comment:
            try:
                btn.click()
            except:
                continue

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source)


In [ ]:
# 將貼文留言爬下做成DataFrame
import numpy as np

divs = soup.find_all('div', class_='tw6a2znq sj5x9vvc d1544ag0 cxgpxx05')

authors = []
comments = []
num_tagged = []
for div in divs:
    author = div.select_one('[class = "d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn mdeji52x e9vueds3 j5wam9gi lrazzd5p oo9gr5id"]')
    authors.append(author.getText())
    comment = div.select('[class = "kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql"]')
    if(len(comment) == 0):
        comments.append('NULL')
    else:
        comments.append(comment[0].getText())
    num = len(div.select('[class = "nc684nl6"]'))
    num_tagged.append(num - 1)
df_authors = pd.DataFrame(authors, columns = ['抽獎人'])
df_comments = pd.DataFrame(comments, columns = ['留言'])
df_num_tagged = pd.DataFrame(num_tagged, columns = ['標記'])
df_result = pd.concat([df_authors, df_comments, df_num_tagged], axis = 1)
df_result

In [ ]:
# 篩選標記人數>=2的抽獎者
result = df_result[df_result['標記'] >= 2].reset_index(drop = True)
result

## 抽獎

In [ ]:
num_prize = int(input("Number of Prizes:"))
prize = []
for i in range(num_prize):
    prize.append(input("Enter prize name. Click enter after finishing each name:"))

In [ ]:
import random
for i in range(num_prize):
    lottery = random.randint(1, len(result))
    display(result.loc[[lottery]])
    print('恭喜', result['抽獎人'][lottery], f'抽中{prize[i]}！')
    result.drop(result.index[lottery]).reset_index(drop = True)